In [ ]:
ord("w") #vede numero unicode associato

In [ ]:
"ciaoo 13%&$/0 彼、元気ですか".encode("utf-8") #cos'i è un byte object, uso utf-8

In [ ]:
list("ciaoo 13%&$/0 彼、元気ですか".encode("utf-8")) # più leggibile mi dà i raw byte

In [ ]:
list("ciaoo ".encode("utf-32")) # nota he con utf-32 ci sono tantissimi 0, in più uno dei motivi per cui è meglio utf-8

# Byte Pair Encoding

## Codice passo passo con esempi

Prendo un testo a caso in questo caso ho preso l'inizio della pagine wikipedia sul pane

In [ ]:
text="Il pane è un prodotto alimentare ottenuto dalla fermentazione, dalla formatura a cui segue una lievitazione, e successiva cottura in forno di un impasto a base di farina (normale o integrale), cereali e acqua, confezionato con diverse modalità, arricchito e caratterizzato frequentemente da ingredienti che si differenziano seguendo le tradizioni locali. Il pane azzimo non è lievitato e si presta alla conservazione per lunghi periodi. Il biscotto del marinaio, detto anche galletta, è diverso, e pure questo adatto a lunga conservazione (anche mesi) ed è tipico della marineria a vela. Si tratta di un bis-cotto, cioè cotto due volte. Anche diversi pani regionali italiani sono azzimi ma è diffuso specialmente in vari paesi medio-orientali, ed è maggiormente prodotto senza aggiunta di sale. Quando ha forma di pane basso è spesso usato come componente di piatti che prevedono la presenza di condimenti (tipici esempi di pane basso regionale italiano sono la pizza napoletana, la focaccia genovese e la piadina romagnola). "

Convertiamo il testo in UTF-8

In [ ]:
tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
print('---')
print(text)
print("length:", len(text))
print('---')
print(tokens)
print("length:", len(tokens)) # sono di più perchè alcuni caratteri, quelli più complessi, hanno bisogno di più byte, ricorda si va fino a 4 byte

Ora faccio un algoritmo che mi trova la coppia di byte più comune

In [ ]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): # Pythonic way to iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(tokens)
#print(stats)

print(sorted(((v,k) for k,v in stats.items()), reverse=True)) # printo in ordine dal più garnde al più piccolo, di base python se ha una lista e chiami la funzione sorted mette in ordine rispetto il primo elemento

Inizializzo count come una variabile dict (dictionary), ex. a={1:"one", (3,4) : 3 , "aaa": "bb"}, il primo è il nome il secondo il valore, puoi richiamarli nel seguente modo: a[1] -> 'one'  ,  a[(3,4)] -> 3  , a["aaa"] -> 'bb' .

La funzione .get per una variabile dict prende il valore associato, nel nostro caso a pair, e se pair non è presente l'aggiunge e mette come valore il valore default che è la seconda entrata che ho messo, i.e. in questo caso 0.

La funzione zip unisce 2 array nel seguente modo: (1,2,3) e (4,5,6) diventa ((1,4), (2,5), (3,6)). Nel caso le dimensioni non cambaciano gli elementi di troppo non vengono calcolati, ex. (1,2,3) , (4,5) diventa ((1,4), (2,5)).

In questo caso la coppia (101, 32) è quella più comune

In [ ]:
chr(101), chr(32)  # vediamo a cosa corrispondono questi caratteri

Ora che abbiamo la coppia più comune diamo a questa coppia l'id 256, poichè i token che ho fino ad ora vanno da 0 a 255. Quindi creo funzione che sostituisce questa coppia con quest'altro token in tutto il testo.

In [ ]:
top_pair = max(stats, key=stats.get) # funzione che mi dà coppia più grande
top_pair

In [ ]:
def merge(ids, pair, idx): # (testo, coppia da sostituire, indice col quale sostituirla)
    # in the list of ints (ids), replace all consecutive occurences of pair with the new token idx
    newids=[]
    i=0
    while i < len(ids):
        # if we are not at the very last position AND the pair matches, replace it
        if i< len(ids) -1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i+=2
        else:
            newids.append(ids[i])
            i+=1
    return newids

# print(merge([5,6,6,7,9,1], (6,7), 99)) # esempio per vedere se funziona
tokens2 = merge(tokens, top_pair, 256)
print(tokens2)
print("length:", len(tokens2))

## Codice generale

In [15]:
text="""Il pane è un prodotto alimentare ottenuto dalla fermentazione, dalla formatura a cui segue una lievitazione, e successiva cottura in forno di un impasto a base di farina (normale o integrale), cereali e acqua, confezionato con diverse modalità, arricchito e caratterizzato frequentemente da ingredienti che si differenziano seguendo le tradizioni locali. Il pane azzimo non è lievitato e si presta alla conservazione per lunghi periodi. Il biscotto del marinaio, detto anche "galletta", è diverso, e pure questo adatto a lunga conservazione (anche mesi) ed è tipico della marineria a vela. Si tratta di un "bis-cotto", cioè cotto due volte. Anche diversi pani regionali italiani sono azzimi ma è diffuso specialmente in vari paesi medio-orientali, ed è maggiormente prodotto senza aggiunta di sale.
Quando ha forma di pane basso è spesso usato come componente di piatti che prevedono la presenza di condimenti (tipici esempi di pane basso regionale italiano sono la pizza napoletana, la focaccia genovese e la piadina romagnola).
Modello con scena di panificazione. primo periodo intermedio dell'Egitto, Dopo il 2090 a.C. Museo Egizio di Torino.
Il pane più antico di cui si abbia certezza risale circa al 12000 a.C. ed è stato ritrovato in Giordania: veniva preparato macinando fra due pietre una miscela di cereali e mescolandola con acqua. L'impasto finale veniva cotto su una pietra rovente.[1]
Intorno al 3000 a.C. gli Egizi perfezionarono la lievitazione, con cui un impasto lasciato all'aria veniva cotto il giorno dopo; ne risultava un pane più soffice e fragrante.[2] Per gli Egizi il pane non era solo una fonte di cibo ma anche di ricchezza.
Gli Ebrei mangiano pane azzimo, Matzah, in occasione della commemorazione dell'esodo dall'Egitto: l'uso del pane non lievitato è simbolo dell'accingersi a intraprendere il viaggio, data la rapidità della preparazione e l'ottima possibilità di conservazione di tale tipo di pane.
In ricordo dell'Ultima Cena di Gesù il pane azzimo, sotto forma di ostie, viene utilizzato nell'Eucaristia da alcune confessioni cristiane (cattolici di rito latino, luterani), mentre altre Chiese (riformate, ortodosse, etc.) utilizzano pane lievitato.
Dall'Egitto l'arte della panificazione passò in Grecia. I greci divennero ottimi panificatori, ne producevano più di 70 qualità. Aggiunsero alle ricette di base ingredienti come: latte, olio, formaggio, erbe aromatiche e miele. Furono anche i primi a preparare il pane di notte. Un tempo nelle campagne ogni famiglia o gruppo di famiglie faceva il pane in casa.
Il pane di frumento è il pane dei paesi occidentali, quindi dell'Europa temperata e della relativa diffusione etnica verso le Americhe delle popolazioni di origine europea.
Ha un posto fondamentale nella tradizione mediterranea come componente primario dell'alimentazione, al punto che il termine stesso può diventare sinonimo di "cibo" o di "nutrimento", non necessariamente fisico. Nella cucina più antica si usava il termine cumpanaticum (oggi companatico) per indicare ogni preparazione che poteva accompagnarsi al pane, sottolineando il suo ruolo fondamentale.
Nei paesi freddi nord europei è spesso diffuso il pane di segale, cereale molto più resistente del frumento al freddo e soprattutto adatto a estati brevi; il pane di segale ha sapore più grezzo del pane di frumento, ma è molto ricco di proteine. La vulnerabilità della segale alla segale cornuta è una delle cause del fuoco di sant'Antonio.tiche e le eventuali denominazioni con il decreto del presidente della Repubblica n. 502 del 30 novembre 1998[3] che modifica la legge n. 580 del 4 luglio 1967[4]. Tale decreto prevede, fra le altre cose, IVA al 4% per il pane normale e quello speciale prodotto con l'aggiunta di burro, olio d'oliva, strutto, latte, zibibbo, uve passe, fichi.
Con altri ingredienti, IVA al 10%. L'umidità del pane destinato al commercio secondo la legge italiana l'art. 16 della Legge 04.07.1967 n. 580 (così modificato dall'art. 22, comma 2, del D.Lgs. 27.01.1992, n. 109) dice: Il contenuto in acqua del pane a cottura completa qualunque sia il tipo di sfarinato impiegato nella produzione del medesimo, con la sola eccezione del pane prodotto con farina integrale, per il quale è consentito un aumento del 2 per cento, è stabilito in base alla pezzatura:
Per le pezzature di peso intermedio, tra quelle sopra indicate, il contenuto massimo in acqua è quello che risulta dalla interpolazione fra i due valori limite.
Le caratteristiche analitiche del pane devono identificarsi coi tipi di farina con i quali il pane è stato prodotto. È tollerata una maggiorazione di 0,05 sul contenuto in ceneri, rispetto a quello degli sfarinati impiegati nella produzione del pane.
Sino al 2006, le aperture di nuovi impianti (panifici) per la produzione del pane erano regolate dalle camere di commercio che rilasciavano una licenza di esercizio. Col Decreto Bersani l'impianto di un nuovo panificio, il trasferimento o la trasformazione di panifici esistenti sono soggetti a una "Dichiarazione di inizio attività". America
In America l'alimentazione corrispondente da carboidrati di base, prima della conquista europea, era data soprattutto dal granturco o mais (Zea mays), in varietà e preparazioni ad alta capacità nutritiva; nelle zone di montagna era ed è presente la Quinoa (Chenopodium quinoa), (pur non essendo questa a rigore un cereale).
Altri alimenti americani ricchi di carboidrati, ma diversi dai cereali e quindi sostanzialmente inadatti alla preparazione del pane erano (e ancora sono) quelli derivati dalla patata, dall'Ulluco e dall'Oxa (od Oca). La citazione è dovuta, dato che l'uso di tali ultimi alimenti è del tutto sostitutiva al pane, rende quindi non necessaria l'alimentazione a base di pane.Pane tipico cileno (sopaipillas) Pane tipico cileno (sopaipillas) Pane tipico colombiano (pandebono)
In Africa e nelle zone calde del sud-ovest asiatico (paesi arabi) spesso è usato il pane di miglio o di sesamo; in specifiche località africane è presente il pane di Teff. L'uso di tali cereali è giustificato dal fatto che in quelle regioni si trovano le condizioni ottimali di coltivazione.Pane tipico egiziano Pane tipico egiziano Pane tradizionale dell'Etiopia (himbasha) Pane tradizionale dell'Etiopia (himbasha) Cottura di un pane tipico algerino (tanguella)Cottura di un pane tipico algerino (tanguella) Asia
Nel Sud-est asiatico e in India, Corea, Cina, e Giappone esiste l'uso di fare "il pane" (o meglio derivati ricchi di carboidrati più o meno analoghi al pane) con il riso, anche in questo caso per precisi motivi climatici che inducono la coltivazione di questo cereale.
Nel continente asiatico si incontrano pani in forme e contenuto diversi, di forma piatta o di panetti solidi compatti o cremosi o gelatinosi, lievitati o non lievitati, ovvero fermentati da batteri acidificanti o trasformati da miceti. Le fermentazioni sono spesso complesse, sorrette da sostanze aggiunte (erbe, fermenti, semi, legumi, proteine da carni o pesce) sulla base di ricette tradizionali gelosamente conservate; le fermentazioni (diversamente dalla lievitazione) hanno una maggiore funzione di arricchimento nutritivo o organolettico, oppure di passaggio di componenti nutritivi importanti (come i derivati di carni o pesce) in un cibo di notevole conservabilità (spesso le fermentazioni acide sono ottimi conservanti). Questo è molto importante in luoghi e situazioni dove i nutrimenti pregiati possono essere molto rari, o non sempre disponibili, e i sistemi di conservazione molto preziosi in ambienti difficili. Il pane quindi diventa solo una base di partenza di un prodotto spesso molto più complesso. Un elemento importantissimo, e a oggi non ancora valorizzato, è l'esame critico dei rendimenti (definibili "notevoli") delle sole trasformazioni, la batterica o la micetica (e non la saccaromicetica) in termini di arricchimento (aumento) in vitamine e proteine, partendo sostanzialmente da semplici carboidrati."""

In [16]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): 
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx): # (testo, coppia da sostituire, indice col quale sostituirla)
    # in the list of ints (ids), replace all consecutive occurences of pair with the new token idx
    newids=[]
    i=0
    while i < len(ids):
        # if we are not at the very last position AND the pair matches, replace it
        if i< len(ids) -1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i+=2
        else:
            newids.append(ids[i])
            i+=1
    return newids

#---
vocab_size = 276 # inserire cardinalità dizionario finale, è un hyper parameter devi scegliere te
num_merges = vocab_size - 255
tokens = text.encode("utf-8") 
tokens = list(map(int, tokens)) 
ids = list(tokens)

merges = {} # (int, int) -> int
for i in range (num_merges):
    stats = get_stats(ids)
    pair = max(stats, key = stats.get)
    idx = 256 + i
    print(f"merging {pair} into a new token {idx}")
    ids = merge(ids, pair, idx)
    merges[pair] = idx
    


merging (101, 32) into a new token 256
merging (105, 32) into a new token 257
merging (111, 32) into a new token 258
merging (97, 32) into a new token 259
merging (111, 110) into a new token 260
merging (97, 110) into a new token 261
merging (97, 108) into a new token 262
merging (101, 110) into a new token 263
merging (101, 114) into a new token 264
merging (44, 32) into a new token 265
merging (97, 116) into a new token 266
merging (100, 257) into a new token 267
merging (105, 110) into a new token 268
merging (105, 99) into a new token 269
merging (101, 108) into a new token 270
merging (263, 116) into a new token 271
merging (122, 105) into a new token 272
merging (112, 261) into a new token 273
merging (101, 115) into a new token 274
merging (97, 114) into a new token 275
merging (116, 258) into a new token 276


In [4]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 8010
ids length: 5909
compression ratio: 1.36X


E' importante sapere che la Tokenizzazione è un processo completamente separato rispetto all'addestramento della rete neurale, si fa su un dataset che potrebbe essere diverso rispetto a quello dell'LLM. Lui ci permette di convertire testo (Unicode code point sequence) in sequenza di tokens e viceversa. Infine LLM non vedrà mai testo ma solo sequenze di token.

## Decoding

Vediamo ora come, data una sequenza di token, risalire al testo corrispondente 

In [ ]:
vocab= {idx: bytes([idx]) for idx in range(256)}
for (p0,p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    # given ids (list of integers), return Python string
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text

# print(decode([128]))

�


Come prima cosa mi creo una specie di dizionario, inizio per ogni token da 1 a 255 gli associo i bytes corrispondenti, poi nel for ci aggiungo pure quelli che ho creato io nel merge. Nota: vocab[p0] + vocab[p1] poichè sono delle variabili di tipo byte non fa la somma algebrica ma li considera tipo stringhe ed unisce semplicemente i byte di uno con quello dell'altro.

La funzione prende in input un array con i vari token che quindi stanno tra 0 e vocab_size, poi crea tokens b"".join(...) mi permette di aggiungere uno dopo l'altro tutte le rappresentazione in byte dei vari token di ids, infatti su tutti gli alementi di ids aggiungo alla variabile che sto costruendo vocab[idx] cioè la rappresentazione in byte corrispondente.

Infine per creare il testo vero e proprio devo convertire questa rappresentazione byte in testo vero e proprio attraverso utf-8. Nota: utf-8 è una rappresentazione particolare, non tutti i byte hanno un valore associato, ad esempio 10000000 (che corrisponde a 128 in base 10) non ha niente associato e se sono sfortunato può succedere che durante questo processo debba affrontare questo token, per questo motivo ho messo errors="replace", questo fa in modo che se incappo in problemi del genere mi mette come valore un simbolo che è una specie di punto interogativo.

## Encoding

In [ ]:
def encode(text):
    #given a string, return list of integer (the tokens)
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break #nothing else can be merged
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

# print(encode("ciao come stai aaaaaa"))

[99, 105, 97, 258, 99, 111, 109, 256, 115, 116, 97, 257, 97, 97, 97, 97, 97, 97]


Prende in input un testo, come prima cosa lo trasforma in bytes con UTF-8, dopodichè se il testo è più lungo di una lettera inzia il processo, se è una stringa nulla o fatta da una lettera non ci sta niente da fare.

Nel while come prima cosa mi richiamo la funzione get_stats che avevo già fatto per avere un dizionario iniziale con le varie coppie, in questo momento non mi interessa quante volte appaiono le varie coppie ma solo quali sono le possibilità.

E' importante che accoppio i bytes seguedo l'ordine di merges quindi dal 256 in poi in modo crescente (poichè quelli successivi potrebbero dipendere da quelli precedenti).
Il min vede tutte le coppie possibili in stats, poi vede queste pair a che valore corrisponde nelle merges (cioè le coppie che ho fatto diventare nuovi token, ogni coppia avrà valore >= 256), se non lo trova mette di default infinito (così non incide nel min), e prende il minimo tra tutti questi questi indici e pair diventa la coppia corrispondente al minimo indice (Questo è un modo inutilmente complicato per fare sta cosa ma è figo). Quindi se questa coppia non sta in merges allora ho finito ed esco dal while, altrimenti pongo idx = token corrispondente alla pair ed uso la funzione già fatta "merge" per sostituire questo token nel testo, ed itero.

 Nota: non sempre se fai decoding(encoding(text)) ottieni lo stesso testo, nella maggiorparte dei casi sì ma potrebbero succedere problemi con utf-8 come già osservato.